# Parsing SEC Filing XBRL Document


## Objective

Parse the filing XBRL file to create a DOM like structure that represent the filing data

## References

* [XBRL Specification - Extensible Business Reporting Language (XBRL) 2.1](https://www.xbrl.org/Specification/XBRL-2.1/REC-2003-12-31/XBRL-2.1-REC-2003-12-31+corrected-errata-2013-02-20.html)

* [List of US GAAP Standards](https://xbrlsite.azurewebsites.net/2019/Prototype/references/us-gaap/)
* [XBRL US - List of Elements](https://xbrl.us/data-rule/dqc_0015-le/)

**Element Version**|**Element ID**|**Namespace**|**Element Label**|**Element Name**|**Balance Type**|**Definition**
:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:
1|1367|us-gaap|Interest Expense|InterestExpense|debit|Amount of the cost of borrowed funds accounted for as interest expense.
2|2692|us-gaap|Cash and Cash Equivalents, at Carrying Value|CashAndCashEquivalentsAtCarryingValue|debit|Amount of currency on hand as well as demand deposits with banks or financial institutions. Includes other kinds of accounts that have the general characteristics of demand deposits. Also includes short-term, highly liquid investments that are both readily convertible to known amounts of cash and so near their maturity that they present insignificant risk of changes in value because of changes in interest rates. Excludes cash and cash equivalents within disposal group and discontinued operation.

## XBRL Element

* [Understanding the Financial Report Logical System](https://www.youtube.com/playlist?list=PLqMZRUzQ64B7EWamzDP-WaYbS_W0RL9nt)
* [XBRL - What is us-gaap:OperatingSegmentsMember element anb where is it defined?](https://money.stackexchange.com/questions/148010/xbrl-what-is-us-gaapoperatingsegmentsmember-element-anb-where-is-it-defined)

### Example
For instance, Qorvo 2020 10K

* [XBRL/rfmd-20210403_htm.xml](https://www.sec.gov/Archives/edgar/data/1604778/000160477821000032/rfmd-20210403_htm.xml)
* [HTML/rfmd-20210403.htm)](https://www.sec.gov/Archives/edgar/data/1604778/000160477821000032/rfmd-20210403.htm):

```
<us-gaap:cashandcashequivalentsatcarryingvalue contextref="*" decimals="-3" id="..." unitref="usd">
  1397880000
</us-gaap:cashandcashequivalentsatcarryingvalue>,
<us-gaap:cashandcashequivalentsatcarryingvalue contextref="***" decimals="-3" id="..." unitref="usd">
  714939000
</us-gaap:cashandcashequivalentsatcarryingvalue>,
<us-gaap:cashandcashequivalentsatcarryingvalue contextref="***" decimals="-3" id="..." unitref="usd">
 711035000
</us-gaap:cashandcashequivalentsatcarryingvalue>
```

Corresponds to the Cash and Cash equivalents in the Cash Flow statement.

<img src="../image/edgar_qorvo_2020_10K_CF.png" align="left" width=800 />

---
# Setup

In [1]:
from typing import (
    List,
    Dict
)
import datetime
import dateutil
import calendar
import re
import requests
import unicodedata
import bs4
from bs4 import BeautifulSoup
from IPython.core.display import (
    display, 
    HTML
)

import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: ('%f' % x).rstrip('0').rstrip('.'))
pd.set_option('display.colheader_justify', 'center')

In [2]:
%%html
<style>
table {float:left}
</style>

In [3]:
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Global Constant

In [4]:
# regexp to extract numeric string
REGEXP_NUMERIC = re.compile(r"\s*[\d.-]*\s*")

re.match(REGEXP_NUMERIC, " -123.12 ")

<re.Match object; span=(0, 9), match=' -123.12 '>

In [5]:
FS_TYPE_10K = "10-K"
FS_TYPE_10Q = "10-Q"

---
# Load EDGAR Filing XBRL

Download the ```_htm.xml``` file from EDGAR. SEC now requires user-agent header.

In [6]:
# AMKOR 2021 10K
CIK = '1047127'
ACCESSION = '000104712721000043'
FS_TYPE = FS_TYPE_10Q

FILING_DIR_URL = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{ACCESSION}"
XBRL_NAME = "amkr-20210930_htm.xml"
XBRL_URL = "/".join([FILING_DIR_URL, XBRL_NAME])

XBRL_URL

'https://www.sec.gov/Archives/edgar/data/1047127/000104712721000043/amkr-20210930_htm.xml'

In [7]:
# OPTICAL CABLE CORPORATION 10-K for the fiscal year ended October 31, 2021
CIK = '0001000230'
ACCESSION = '000143774921028951'
FS_TYPE = FS_TYPE_10K

FILING_DIR_URL = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{ACCESSION}"
XBRL_NAME = "occ20211031_10k_htm.xml"
XBRL_URL = "/".join([FILING_DIR_URL, XBRL_NAME])

XBRL_URL

'https://www.sec.gov/Archives/edgar/data/0001000230/000143774921028951/occ20211031_10k_htm.xml'

In [8]:
# QORVO 2021 10K
CIK = '1604778'
ACCESSION = '000160477821000032'
FS_TYPE = FS_TYPE_10K

FILING_DIR_URL = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{ACCESSION}"
XBRL_NAME = "rfmd-20210403_htm.xml"
XBRL_URL = "/".join([FILING_DIR_URL, XBRL_NAME])

XBRL_URL

'https://www.sec.gov/Archives/edgar/data/1604778/000160477821000032/rfmd-20210403_htm.xml'

In [9]:
# DIOD 2020 10K
CIK = '29002'
ACCESSION = '000156459021007008'
FS_TYPE = FS_TYPE_10K

FILING_DIR_URL = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{ACCESSION}"
XBRL_NAME = "diod-10k_20201231_htm.xml"
XBRL_URL = "/".join([FILING_DIR_URL, XBRL_NAME])

XBRL_URL

'https://www.sec.gov/Archives/edgar/data/29002/000156459021007008/diod-10k_20201231_htm.xml'

In [10]:
headers = {"User-Agent": "Company Name myname@company.com"}
response = requests.get(XBRL_URL, headers=headers)

if response.status_code == 200:
    content = response.content.decode("utf-8") 
else:
    assert False, f"{XBRL_URL} failed with status {response.status_code}"

In [11]:
soup = BeautifulSoup(content, 'html.parser')

## Repoting period

Each 10-K and 10-Q XBRL has the reporting period for the filing. To exclude the other period, e.g. pervious year or quarter, use the ```context id``` for the reporting period. **Most** 10-K, 10-Q specify the annual period with the first ```<startDate> and <endDate>``` tags.

For instances:

### QRVO 10-K 2020

```
<context id="ifb6ce67cf6954ebf88471dd82daa9247_D20200329-20210403">
    <entity>
    <identifier scheme="http://www.sec.gov/CIK">0001604778</identifier>
    </entity>
    <period>
        <startDate>2020-03-29</startDate>
        <endDate>2021-04-03</endDate>
    </period>
</context>
```

### AMKR 10-K 2020

```
<context id="i5fac0a392353427b8266f185495754d3_D20200101-20201231">
    <entity>
    <identifier scheme="http://www.sec.gov/CIK">0001047127</identifier>
    </entity>
    <period>
        <startDate>2020-01-01</startDate>
        <endDate>2020-12-31</endDate>
    </period>
</context>
```

### AAPL 10-Q 4th QTR 2020

```
<context id="i6e431846933d461fb8c8c0bdf98c9758_D20200927-20201226">
    <entity>
    <identifier scheme="http://www.sec.gov/CIK">0000320193</identifier>
    </entity>
    <period>
        <startDate>2020-09-27</startDate>
        <endDate>2020-12-26</endDate>
    </period>
</context>
```

**However, there are companies that do not have this manner**. For instance [10-K for OPTICAL CABLE CORPORATION(CIK=0001000230)](https://www.sec.gov/Archives/edgar/data/1000230/000143774921028951/occ20211031_10k_htm.xml) has the same start and end dates at first.

```
<context id="d202110K">
	<entity>
		<identifier scheme="http://www.sec.gov/CIK">0001000230</identifier>
	</entity>
	<period>
		<startDate>2021-10-31</startDate>   # <-----
		<endDate>2021-10-31</endDate>
	</period>
</context>
<context id="d_2020-11-01_2021-10-31">
	<entity>
		<identifier scheme="http://www.sec.gov/CIK">0001000230</identifier>
	</entity>
	<period>
		<startDate>2020-11-01</startDate>   # <-----
		<endDate>2021-10-31</endDate>
	</period>
</context>
```

The report uses the 2nd for 2021 F/S element value but does not use the first one.

**B/S**
```
<us-gaap:Assets 
    contextRef="i_2021-10-31"    # <-----
    decimals="INF" 
    id="c79893606" 
    unitRef="USD"
>
  37916530
</us-gaap:Assets>
```

**P/L**
```
<us-gaap:RevenueFromContractWithCustomerIncludingAssessedTax 
  contextRef="d_2020-11-01_2021-10-31"     # <-----
  decimals="INF" 
  id="c79893662" 
  unitRef="USD"
>
  59136294
</us-gaap:RevenueFromContractWithCustomerIncludingAssessedTax>
```

<img src='../image/edgar_optical_cable_2021_10K.png' align="left" width=500/>

### Get the period from the 1st

For now, just get the period from the 1st **period** element.

In [12]:
def get_period(soup):
    """Identify the report period (end date) from the first context tag in the statement.
    <context id="ifb6ce67cf6954ebf88471dd82daa9247_D20200329-20210403">
        <entity>
        <identifier scheme="http://www.sec.gov/CIK">0001604778</identifier>
        </entity>
        <period>
            <startDate>2020-03-29</startDate>
            <endDate>2021-04-03</endDate>        # <-----
        </period>
    </context>
    Returns: reporting period e.g. "2021-09-30"
    """
    first_context = None
    for context in soup.find_all('context'):
        if context.find('period') and context.find('period').find('enddate'):
            first_context = context
            break      
        
    assert first_context is not None, "No period found"
    period = first_context.find('period').find('enddate').text.strip()
    
    try:
        dateutil.parser.parse(period)
    except ValueError as e:
        assert False, f"Invalid period {period} found."
            
    return period

In [13]:
PERIOD = get_period(soup)
PERIOD

'2020-12-31'

### Regexp to find all the contexts that match with PERIOD

10-K, 10-Q F/S uses multiple contexts to refer to the F/S element values for the **period**. Collect all the contexts for the **period**.

In [14]:
def get_target_contexts(soup, period, type=FS_TYPE_10K):
    target_contexts = []

    end_date = dateutil.parser.parse(period)
    duration = datetime.timedelta(days=365) if type == FS_TYPE_10K else datetime.timedelta(days=90)

    # The StartDate of the valid context is within +/- 30 days of the starting date of the period.
    # If the report is 10-K and the period end is 2021-10-31, then the validate date is between
    # 2020-10-1 and 2020-11-1
    from_date = end_date - duration - datetime.timedelta(days=30)
    to_date = end_date - duration + datetime.timedelta(days=30)

    # Contexts with startdate/enddate
    for context in soup.find_all('context'):
        if context.find('period') and context.find('period').find('enddate', string=period):
            start = dateutil.parser.parse(context.find('period').find('startdate').text)
            if start == end_date or (from_date < start and start < to_date):
                target_contexts.append(context)

    
    # Contexts with 'instant' element
    target_contexts.extend([
        context for context in soup.find_all('context') 
        if context.find('period') and context.find('period').find(['instant'], string=PERIOD)
    ])
    
    return target_contexts

In [15]:
# get_target_contexts(soup, PERIOD)

In [16]:
def get_target_context_ids(soup, period, type=FS_TYPE_10K):
    target_contexts = []

    end_date = dateutil.parser.parse(period)
    duration = datetime.timedelta(days=365) if type == FS_TYPE_10K else datetime.timedelta(days=90)

    # The StartDate of the valid context is within +/- 30 days of the starting date of the period.
    # If the report is 10-K and the period end is 2021-10-31, then the validate date is between
    # 2020-10-1 and 2020-11-1
    from_date = end_date - duration - datetime.timedelta(days=30)
    to_date = end_date - duration + datetime.timedelta(days=30)

    # Contexts with startdate/enddate
    for context in soup.find_all('context'):
        # print("-"*100)
        # print(context)
        if context.find('period') and context.find('period').find('enddate', string=period):
            start = dateutil.parser.parse(context.find('period').find('startdate').text)
            if start == end_date or (from_date < start and start < to_date):
                target_contexts.append(context['id'])

    
    # Contexts with 'instant' element
    target_contexts.extend([
        context['id'] for context in soup.find_all('context') 
        if context.find('period') and context.find('period').find(['instant'], string=PERIOD)
    ])
    
    return target_contexts

# Constant

In [17]:
# XBRL Namespace
NAMESPACE = "us-gaap"

In [18]:
ELEMENT_TYPE_DEBIT = 'debit'
ELEMENT_TYPE_CREDIT = 'credit'
ELEMENT_TYPE_CALC = 'calc'      # calculated value e.g. sub total 
ELEMENT_TYPE_METRIC = 'metric'  # metrics e.g. EPS
ELEMENT_TYPE_FACT = 'fact'

CREDIT_ITEMS = set(map(str.lower, [
    # P/L
    f"{NAMESPACE}:Revenues",
    f"{NAMESPACE}:RevenueFromContractWithCustomerExcludingAssessedTax",
    f"{NAMESPACE}:DeferredRevenue",
    f"{NAMESPACE}:RevenueMineralSales",
    f"{NAMESPACE}:DeferredRevenueCurrent",
    f"{NAMESPACE}:HealthCareOrganizationRevenue",
    f"{NAMESPACE}:SalesRevenueGoodsGross",
    f"{NAMESPACE}:RealEstateRevenueNet",
    f"{NAMESPACE}:SalesOfRealEstate",
    f"{NAMESPACE}:AdvertisingRevenue",
    f"{NAMESPACE}:ElectricalGenerationRevenue",
    f"{NAMESPACE}:ContractsRevenue",
    f"{NAMESPACE}:AdmissionsRevenue",
    f"{NAMESPACE}:InsuranceServicesRevenue",
    f"{NAMESPACE}:BrokerageCommissionsRevenue",
    f"{NAMESPACE}:OtherNonoperatingIncomeExpense",  # Why this is in credit?
    f"{NAMESPACE}:CashAndCashEquivalentsAtCarryingValue",
    f"{NAMESPACE}:OtherAlternativeEnergySalesRevenue",
    f"{NAMESPACE}:RegulatedAndUnregulatedOperatingRevenue",
    
    # B/S (Assets)
    f"{NAMESPACE}:RestrictedCashEquivalentsCurrent",
    f"{NAMESPACE}:ShortTermInvestments",
    f"{NAMESPACE}:ReceivablesNetCurrent",
    f"{NAMESPACE}:InventoryNet",
    f"{NAMESPACE}:PrepaidExpenseAndOtherAssetsCurrent",
    f"{NAMESPACE}:PropertyPlantAndEquipmentNet",
    f"{NAMESPACE}:DeferredIncomeTaxAssetsNet",
    f"{NAMESPACE}:GoodWill",
    f"{NAMESPACE}:IntangibleAssetsNetExcludingGoodwill",
    f"{NAMESPACE}:OtherAssetsNoncurrent",
    
    
]))

DEBIT_ITEMS = set(map(str.lower, [
    # P/L
    f"{NAMESPACE}:CostOfRevenue",
    f"{NAMESPACE}:CostOfGoods",
    f"{NAMESPACE}:CostOfGoodsAndServicesSold",
    f"{NAMESPACE}:CostOfServicesLicensesAndServices",
    f"{NAMESPACE}:ContractRevenueCost",
    f"{NAMESPACE}:LicenseCosts",
    f"{NAMESPACE}:ResearchAndDevelopmentExpense",
    f"{NAMESPACE}:SellingGeneralAndAdministrativeExpense",
    f"{NAMESPACE}:OtherCostAndExpenseOperating",
    f"{NAMESPACE}:IncomeTaxExpenseBenefit",
    f"{NAMESPACE}:InterestExpense",
    
    # B/S (Liabilities)
    f"{NAMESPACE}:AccountsPayableCurrent",
    f"{NAMESPACE}:AccruedLiabilitiesCurrent",
    f"{NAMESPACE}:TaxesPayableCurrent",
    f"{NAMESPACE}:LongTermDebtCurrent",
    f"{NAMESPACE}:LongTermLoansFromBank",
    f"{NAMESPACE}:DeferredIncomeTaxLiabilitiesNet",
    f"{NAMESPACE}:OtherLiabilitiesNoncurrent",
    # B/S (SE)
    f"{NAMESPACE}:AdditionalPaidInCapitalCommonStock",
    f"{NAMESPACE}:RetainedEarningsAppropriated",
    f"{NAMESPACE}:AccumulatedOtherComprehensiveIncomeLossNetOfTax",
    f"{NAMESPACE}:MinorityInterest",
    
]))

CALC_ITEMS = set(map(str.lower, [
    # P/L
    f"{NAMESPACE}:OperatingExpenses",
    f"{NAMESPACE}:GrossProfit",
    f"{NAMESPACE}:OperatingIncomeLoss",
    f"{NAMESPACE}:NetIncomeLoss",    

    # B/S (Credit)
    f"{NAMESPACE}:AssetsCurrent",    
    f"{NAMESPACE}:Assets",    
    # B/S (Debit/Liabilities)
    f"{NAMESPACE}:LiabilitiesCurrent",
    f"{NAMESPACE}:Liabilities",
    # B/S (Debit/SE)
    f"{NAMESPACE}:StockholdersEquity",
    f"{NAMESPACE}:StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest",
    f"{NAMESPACE}:LiabilitiesAndStockholdersEquity",
    
]))

METRIC_ITEMS = set(map(str.lower, [
    f"{NAMESPACE}:EarningsPerShareBasic",
    f"{NAMESPACE}:EarningsPerShareBasicAndDiluted",
]))

FACT_ITEMS = set(map(str.lower, [
    f"{NAMESPACE}:SharesOutstanding",
    f"{NAMESPACE}:CommonStockSharesOutstanding",
    f"{NAMESPACE}:CommonStockOtherSharesOutstanding",
]))

In [19]:
CONTEXT_REGEXP = re.compile("|".join(get_target_context_ids(soup=soup, period=PERIOD, type=FS_TYPE)))

# Utilities

In [20]:
def assert_bf4_tag(element):
    assert isinstance(element, bs4.element.Tag), \
    f"Expected BS4 tag but {element} of type {type(element)}"

In [21]:
def display_elements(elements):
    assert isinstance(elements, bs4.element.ResultSet) or isinstance(elements[0], bs4.element.Tag)
    for element in elements: # decimals="-3" means the displayed value is divied by 1000.
        print(f"{element.name:80} {element['unitref']:5} {element['decimals']:5} {element.text:15}")

In [22]:
def get_element_hash(element):
    """Generate the financial element hash key to uniquely identify an financial element record
    In a F/S, the same element, e.g. gaap:CashAndCashEquivalentsAtCarryingValue can be used at
    multiple places, one in B/S and one in P/L.
    
    To be able to identify if two elements are the same, provides a way to be able to compare
    two elements by generating a hash key from the attributes of an element.
    
    Args:
        element: bs4.element.Tag for an financial element
    Returns: hash key
    """
    assert isinstance(element, bs4.element.Tag)
    # key = f"{element.name}{element['unitref']}{element['contextref']}{element.text}"
    # key = f"{element.name}{element['unitref']}{element.text}"
    key = f"{element.name}{element['unitref']}"
    return hash(key)

In [23]:
# XBRL attribute conditions to match when extracting FS elements
ATTRIBUTES = {
    "contextref": CONTEXT_REGEXP,
    "decimals": True, 
    "unitref": True
}


def find_financial_elements(soup, element_names, attributes=ATTRIBUTES):
    """Find the financial statement elements from the XML/HTML source.
    Args:
        soup: BS4 source
        element_names: String or regexp instance to select the financial elements.
        attribute: tag attributes to select the financial elements
    Returns:
        List of BS4 tag objects that matched the element_names and attributes.
    """
    assert isinstance(soup, BeautifulSoup)
    assert isinstance(element_names, re.Pattern) or isinstance(element_names, str)

    
    names = element_names.lower() if isinstance(element_names, str) else element_names

    elements = soup.find_all(
        name=names,
        string=REGEXP_NUMERIC,
        attrs=attributes
    )
    
    # Select unique elements
    hashes = set([]) 
    results = []
    if elements is not None and len(elements) > 0:
        for element in elements:
            hash_value = get_element_hash(element)
            if hash_value not in hashes:
                results.append(element) 
                hashes.add(hash_value)

    return results

In [24]:
for element in find_financial_elements(soup, re.compile(f"^{NAMESPACE}:.*"), attributes=ATTRIBUTES):
    print(element.name)

us-gaap:commonstockparorstatedvaluepershare
us-gaap:sharebasedcompensationarrangementbysharebasedpaymentawardoptionsexercisablenumber
us-gaap:sharebasedcompensationarrangementbysharebasedpaymentawardoptionsexercisableweightedaverageexerciseprice
us-gaap:sharebasedcompensationarrangementbysharebasedpaymentawardoptionsexercisableintrinsicvalue1
us-gaap:sharebasedcompensationsharesauthorizedunderstockoptionplansexercisepricerangeupperrangelimit
us-gaap:sharebasedcompensationsharesauthorizedunderstockoptionplansexercisepricerangelowerrangelimit
us-gaap:cashandcashequivalentsatcarryingvalue
us-gaap:restrictedcash
us-gaap:shortterminvestments
us-gaap:allowancefornotesandloansreceivablecurrent
us-gaap:receivablesnetcurrent
us-gaap:inventorynet
us-gaap:prepaidexpenseandotherassetscurrent
us-gaap:assetscurrent
us-gaap:propertyplantandequipmentnet
us-gaap:deferredincometaxassetsnet
us-gaap:goodwill
us-gaap:intangibleassetsnetexcludinggoodwill
us-gaap:otherassetsnoncurrent
us-gaap:assets
us-gaap:

In [25]:
def get_financial_element_numeric_values(elements):
    # assert isinstance(elements, bs4.element.ResultSet) or isinstance(elements[0], bs4.element.Tag)
    assert_bf4_tag(elements[0])
    
    values = []
    for element in elements:
        assert re.match(REGEXP_NUMERIC, element.text.strip()), f"Element must be numeric but {element.text}"
        values.append(float(element.text))
        
    return values

In [26]:
def get_financial_element_columns():
    """Financial record columns"""
    return ["type", "name", "unit", "decimals", "value", "context"]

def get_record_for_nil_elements(elements, element_type=None):
    results = []
    for element in elements:
        # Type of the elemen
        if element_type is None and element.name in DEBIT_ITEMS: element_type = ELEMENT_TYPE_DEBIT
        if element_type is None and element.name in CREDIT_ITEMS: element_type = ELEMENT_TYPE_CREDIT
        if element_type is None and element.name in CALC_ITEMS: element_type = ELEMENT_TYPE_CALC
        if element_type is None and element.name in METRIC_ITEMS: element_type = ELEMENT_TYPE_METRIC
        if element_type is None and element.name in FACT_ITEMS: element_type = ELEMENT_TYPE_FACT

        # Name of the financial element
        element_name = element.name

        # Unit of the financial element
        element_unit = element['unitref']

        # Scale of the element
        element_scale = int(element['decimals']) if element['decimals'].lower() != 'inf' else np.inf

        # Value of the element
        element_value = -np.inf

        # Context ID of the element
        element_context = element['contextref']

        results.append([
            element_type,
            element_name,
            element_unit, 
            element_scale, 
            element_value, 
            element_context
        ])
        
    return results

def get_records_for_financial_elements(elements, element_type=None):
    """Financial record having the columns of get_financial_element_columns"""
    # assert isinstance(elements, bs4.element.ResultSet) or isinstance(elements[0], bs4.element.Tag)
    assert_bf4_tag(elements[0])
    
    results = []
    for element in elements:
        # Type of the elemen
        if element_type is None and element.name in DEBIT_ITEMS: element_type = ELEMENT_TYPE_DEBIT
        if element_type is None and element.name in CREDIT_ITEMS: element_type = ELEMENT_TYPE_CREDIT
        if element_type is None and element.name in CALC_ITEMS: element_type = ELEMENT_TYPE_CALC
        if element_type is None and element.name in METRIC_ITEMS: element_type = ELEMENT_TYPE_METRIC
        if element_type is None and element.name in FACT_ITEMS: element_type = ELEMENT_TYPE_FACT
        
        # Name of the financial element
        element_name = element.name
        
        # Unit of the financial element
        element_unit = element['unitref']
        
        # Scale of the element
        element_scale = int(element['decimals']) if element['decimals'].lower() != 'inf' else np.inf
            
        # Value of the element
        element_value = float(element.text)

        # Context ID of the element
        element_context = element['contextref']
        
        results.append([
            element_type,
            element_name,
            element_unit, 
            element_scale, 
            element_value, 
            element_context
        ])
        
    return results

In [27]:
def get_records_for_financial_element_names(soup, names: List[str], element_type=None):
    """Get finacial records that matches the financial element names
    """
    elements = find_financial_elements(soup=soup, element_names=names)
    if len(elements) > 0:
        display_elements(elements)
        return get_records_for_financial_elements(elements, element_type)
    else:
        return get_record_for_nil_elements(elements)

In [28]:
def get_values_for_financial_element_names(soup, names: List[str]):
    elements = find_financial_elements(soup=soup, element_names=names)
    if len(elements) > 0:
        display_elements(elements)
        return get_financial_element_numeric_values(elements)
    else:
        return []

---
# Shares Outstanding

In [29]:
def get_shared_outstanding(soup):
    names = re.compile("|".join([
        rf"{NAMESPACE}:SharesOutstanding",
        rf"{NAMESPACE}:CommonStockSharesOutstanding",
        rf"{NAMESPACE}:CommonStockOtherSharesOutstanding",
    ]).lower())

    return get_records_for_financial_element_names(soup=soup, names=names)

In [30]:
shares_outstandings = get_shared_outstanding(soup)
shares_outstandings

us-gaap:commonstocksharesoutstanding                                             U_xbrlishares INF   44276194       


[['fact',
  'us-gaap:commonstocksharesoutstanding',
  'U_xbrlishares',
  inf,
  44276194.0,
  'C_0000029002_20201231']]

In [31]:
df_ShareOutstanding = pd.DataFrame(shares_outstandings)
df_ShareOutstanding

,0,1,2,3,4,5
0,fact,us-gaap:commonstocksharesoutstanding,U_xbrlishares,inf,44276194,C_0000029002_20201231


---
# Statements of Income (P/L)

In [32]:
PL = []

## Revenues

In [33]:
def get_revenues(soup):
    names = rf"{NAMESPACE}:Revenues".lower()
    names = re.compile("|".join([
        rf"^{NAMESPACE}:Revenues$",
        rf"^{NAMESPACE}:RevenueFromContractWithCustomerExcludingAssessedTax$",
        rf"^{NAMESPACE}:DeferredRevenue$",
        rf"^{NAMESPACE}:RealEstateRevenueNet$",
        rf"^{NAMESPACE}:SalesOfRealEstate$",
        rf"^{NAMESPACE}:AdvertisingRevenue$",
        rf"^{NAMESPACE}:ElectricalGenerationRevenue$",
        rf"^{NAMESPACE}:ContractsRevenue$",
        rf"^{NAMESPACE}:RevenueMineralSales$",
        rf"^{NAMESPACE}:DeferredRevenueCurrent$",
        rf"^{NAMESPACE}:HealthCareOrganizationRevenue$",
        rf"^{NAMESPACE}:SalesRevenueGoodsGross$",
        rf"^{NAMESPACE}:AdmissionsRevenue$",
        rf"^{NAMESPACE}:InsuranceServicesRevenue$",
        rf"^{NAMESPACE}:BrokerageCommissionsRevenue$",
        rf"^{NAMESPACE}:RegulatedAndUnregulatedOperatingRevenue$"
        rf"^{NAMESPACE}:OtherAlternativeEnergySalesRevenue$",
    ]).lower())    
    
    return get_records_for_financial_element_names(soup=soup, names=names)
    

In [34]:
PL += get_revenues(soup)

us-gaap:revenuefromcontractwithcustomerexcludingassessedtax                      U_iso4217USD -3    1229215000     


## Cost of Revenues

In [35]:
def get_cost_of_revenues(soup):
    names = re.compile("|".join([
        rf"^{NAMESPACE}:CostOfRevenue$",
        rf"^{NAMESPACE}:CostOfGoods$",
        rf"^{NAMESPACE}:CostOfGoodsAndServicesSold$",
        rf"^{NAMESPACE}:ContractRevenueCost$",
        rf"^{NAMESPACE}:CostOfServicesLicensesAndServices$"
        rf"^{NAMESPACE}:LicenseCosts$"
    ]).lower())
    return get_records_for_financial_element_names(soup=soup, names=names)

In [36]:
PL += get_cost_of_revenues(soup)

us-gaap:costofgoodsandservicessold                                               U_iso4217USD -3    798094000      


## ***___# Gross Profit___***

In [37]:
def get_gross_profit(soup):
    names = f"{NAMESPACE}:GrossProfit".lower()
    return get_records_for_financial_element_names(soup=soup, names=names)

In [38]:
PL += get_gross_profit(soup)

us-gaap:grossprofit                                                              U_iso4217USD -3    431121000      


## Operating Expenses

### Research and Development

In [39]:
names = f"{NAMESPACE}:ResearchAndDevelopmentExpense".lower()
PL += get_records_for_financial_element_names(soup, names)

us-gaap:researchanddevelopmentexpense                                            U_iso4217USD -3    94288000       


### Administrative Expense

In [40]:
names = f"{NAMESPACE}:SellingGeneralAndAdministrativeExpense"
PL += get_records_for_financial_element_names(soup, names)

us-gaap:sellinggeneralandadministrativeexpense                                   U_iso4217USD -3    185067000      


### Other operating expenses

In [41]:
find_financial_elements(soup=soup, element_names=names)

[<us-gaap:sellinggeneralandadministrativeexpense contextref="C_0000029002_20200101_20201231" decimals="-3" id="F_000133" unitref="U_iso4217USD">185067000</us-gaap:sellinggeneralandadministrativeexpense>]

In [42]:
names = f"{NAMESPACE}:OtherCostAndExpenseOperating"
PL += get_records_for_financial_element_names(soup=soup, names=names)

## ***___# Total Operating Expenses___***

In [43]:
names = f"{NAMESPACE}:OperatingExpenses".lower()
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:operatingexpenses                                                        U_iso4217USD -3    296789000      


## ***___# Operating Income___***

$GrossProfit - Total Operating Expenses$

In [44]:
def get_operating_income(soup):
    names = f"{NAMESPACE}:OperatingIncomeLoss".lower()
    return get_records_for_financial_element_names(soup=soup, names=names)

In [45]:
PL += get_operating_income(soup)

us-gaap:operatingincomeloss                                                      U_iso4217USD -3    134332000      


## Other Expenses

### Interest Expense

* [Investopedia - What Is an Interest Expense?](https://www.investopedia.com/terms/i/interestexpense.asp)

> An interest expense is the cost incurred by an entity for borrowed funds. Interest expense is a non-operating expense shown on the income statement. It represents interest payable on any borrowings – bonds, loans, convertible debt or lines of credit. It is essentially calculated as the interest rate times the outstanding principal amount of the debt. Interest expense on the income statement represents ***interest accrued during the period*** covered by the financial statements, and **NOT the amount of interest paid over that period**. While interest expense is tax-deductible for companies, in an individual's case, it depends on his or her jurisdiction and also on the loan's purpose.  
>
> For most people, mortgage interest is the single-biggest category of interest expense over their lifetimes as interest can total tens of thousands of dollars over the life of a mortgage as illustrated by online calculators.

In [46]:
names = f"{NAMESPACE}:InterestExpense"
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:interestexpense                                                          U_iso4217USD -3    11662000       


### Other Non-operating Expenses

In [47]:
names = f"{NAMESPACE}:OtherNonoperatingIncomeExpense"
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:othernonoperatingincomeexpense                                           U_iso4217USD -3    6419000        


## Income Tax

In [48]:
names = f"{NAMESPACE}:IncomeTaxExpenseBenefit"
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:incometaxexpensebenefit                                                  U_iso4217USD -3    21112000       


## ***___# Net Income___***

$GrossProfit - (Operating Expenses + NonOperating Expense) - Tax$

In [49]:
def get_net_income(soup):
    names = f"{NAMESPACE}:NetIncomeLoss".lower()
    return get_records_for_financial_element_names(soup=soup, names=names)

In [50]:
PL += get_net_income(soup)

us-gaap:netincomeloss                                                            U_iso4217USD -3    98088000       


## ***___# Shares Outstandings___***

In [51]:
PL += shares_outstandings

## ***___# Net Income Per Share___***

* [US GAAP - Is Net Income Per Share the same with EPS?](https://money.stackexchange.com/questions/148015/us-gaap-is-net-income-per-share-the-same-with-eps)

In [52]:
def get_eps(soup):
    return get_records_for_financial_element_names(
        soup=soup, names=f"{NAMESPACE}:EarningsPerShareBasic".lower()
    ) + \
    get_records_for_financial_element_names(
        soup=soup, names=f"{NAMESPACE}:EarningsPerShareBasicAndDiluted".lower()
    )

In [53]:
PL += get_eps(soup)

us-gaap:earningspersharebasic                                                    U_iso4217USD_xbrlishares 2     1.92           


## Display P/L

Is ```us-gaap:othernonoperatingincomeexpense``` credit or debit? As the value is **negative** and so is in the Income Statement, is shoudl be credit -> To be confirmed. 

In [54]:
df_PL = pd.DataFrame(PL, columns=get_financial_element_columns())
df_PL

,type,name,unit,decimals,value,context
0,credit,us-gaap:revenuefromcontractwithcustomerexcludi...,U_iso4217USD,-3,1229215000,C_0000029002_20200101_20201231
1,debit,us-gaap:costofgoodsandservicessold,U_iso4217USD,-3,798094000,C_0000029002_20200101_20201231
2,calc,us-gaap:grossprofit,U_iso4217USD,-3,431121000,C_0000029002_20200101_20201231
3,debit,us-gaap:researchanddevelopmentexpense,U_iso4217USD,-3,94288000,C_0000029002_20200101_20201231
4,debit,us-gaap:sellinggeneralandadministrativeexpense,U_iso4217USD,-3,185067000,C_0000029002_20200101_20201231
5,calc,us-gaap:operatingexpenses,U_iso4217USD,-3,296789000,C_0000029002_20200101_20201231
6,calc,us-gaap:operatingincomeloss,U_iso4217USD,-3,134332000,C_0000029002_20200101_20201231
7,debit,us-gaap:interestexpense,U_iso4217USD,-3,11662000,C_0000029002_20200101_20201231
8,credit,us-gaap:othernonoperatingincomeexpense,U_iso4217USD,-3,6419000,C_0000029002_20200101_20201231
9,debit,us-gaap:incometaxexpensebenefit,U_iso4217USD,-3,21112000,C_0000029002_20200101_20201231


In [55]:
credits = df_PL[df_PL['type'] == 'credit']['value'].sum()
credits

1235634000.0

In [56]:
debits = df_PL[df_PL['type'] == 'debit']['value'].sum()
debits

1110223000.0

In [57]:
credits - debits  # Equal to the Net Income

125411000.0

In [58]:
eps = np.divide(
    df_PL[df_PL['name'] == 'us-gaap:netincomeloss']['value'].values,
    df_PL[df_PL['name'] == 'us-gaap:commonstocksharesoutstanding']['value'].values
).item()
scale = 2

print(f"{eps:.{scale}f}")

2.22


---
# Balance Sheet (B/S)

In [59]:
BS = []

## Cash & Cash Equivalents

Look for the cash and cash equivalents for the reporting perid in the Balance Sheet and Cash Flow statements of the  10-K.

In [60]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:CashAndCashEquivalentsAtCarryingValue$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:cashandcashequivalentsatcarryingvalue                                    U_iso4217USD -3    268065000      


## Restricted Cash

In [61]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:RestrictedCashEquivalentsCurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:restrictedcashequivalentscurrent                                         U_iso4217USD -3    52464000       


## Short Term Investments

In [62]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:ShortTermInvestments$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:shortterminvestments                                                     U_iso4217USD -3    6142000        


## Account Receivable

In [63]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:ReceivablesNetCurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:receivablesnetcurrent                                                    U_iso4217USD -3    320061000      


## ***___Inventory___***

In [64]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:InventoryNet$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:inventorynet                                                             U_iso4217USD -3    307062000      


## Prepaid Expense / Other Assets Current

* [Understanding Prepaid Expenses](https://www.investopedia.com/terms/p/prepaidexpense.asp)

> Companies make prepayments for goods or services such as leased office equipment or insurance coverage that provide continual benefits over time. Goods or services of this nature cannot be expensed immediately because the expense would not line up with the benefit incurred over time from using the asset.  
>
> According to generally accepted accounting principles (GAAP), expenses should be recorded in the same accounting period as the benefit generated from the related asset.

* [us-gaap: PrepaidExpenseAndOtherAssetsCurrent](https://www.calcbench.com/element/PrepaidExpenseAndOtherAssetsCurrent)

> Amount of asset related to consideration paid in advance for costs that provide economic benefits in future periods, and amount of other assets that are expected to be realized or consumed within one year or the normal operating cycle, if longer.

* [Other Current Assets (OCA)](https://www.investopedia.com/terms/o/othercurrentassets.asp)

> Other current assets (OCA) is a category of things of value that a company owns, benefits from, or uses to generate income that can be converted into cash within one business cycle. They are referred to as “other” because they are uncommon or insignificant, unlike typical current asset items such as cash, securities, accounts receivable, inventory, and prepaid expenses.

In [65]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:PrepaidExpenseAndOtherAssetsCurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:prepaidexpenseandotherassetscurrent                                      U_iso4217USD -3    70193000       


## ***___# Current Assets ---***

In [66]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:AssetsCurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:assetscurrent                                                            U_iso4217USD -3    1023987000     


## Property, Plant, Equipment

In [67]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:PropertyPlantAndEquipmentNet$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:propertyplantandequipmentnet                                             U_iso4217USD -3    530815000      


## Deferred Tax

In [68]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:DeferredIncomeTaxAssetsNet$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:deferredincometaxassetsnet                                               U_iso4217USD -3    57841000       


## ***___GoodWill___***

In [69]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:GoodWill$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:goodwill                                                                 U_iso4217USD -3    158331000      


## Intangible and Other Assets

In [70]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:IntangibleAssetsNetExcludingGoodwill$",
    rf"^{NAMESPACE}:OtherAssetsNoncurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:intangibleassetsnetexcludinggoodwill                                     U_iso4217USD -3    110591000      
us-gaap:otherassetsnoncurrent                                                    U_iso4217USD -3    97892000       


## ***___# Total Assets___***

In [71]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:Assets$",
]).lower())

BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:assets                                                                   U_iso4217USD -3    1979457000     


## Account Payable

* [Accounts Payable (AP)](https://www.investopedia.com/terms/a/accountspayable.asp)

> company's obligation to pay off a short-term debt to its creditors or suppliers.



* [Accrued Liability](https://www.investopedia.com/terms/a/accrued-liability.asp) (売掛金)

> costs for goods and services already delivered to a company for which it must pay in the future. A company can accrue liabilities for any number of obligations and are recorded on the company's balance sheet. They are normally listed on the balance sheet as current liabilities and are adjusted at the end of an accounting period.


* [us-gaap:AccruedLiabilitiesCurrent](http://xbrlsite.azurewebsites.net/2019/Prototype/references/us-gaap/Element-354.html)

In [72]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:AccountsPayableCurrent$",
    rf"^{NAMESPACE}:AccruedLiabilitiesCurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:accountspayablecurrent                                                   U_iso4217USD -3    168045000      
us-gaap:accruedliabilitiescurrent                                                U_iso4217USD -3    160117000      


## Tax

In [73]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:TaxesPayableCurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:taxespayablecurrent                                                      U_iso4217USD -3    19177000       


## Debt Due

In [74]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:LongTermDebtCurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:longtermdebtcurrent                                                      U_iso4217USD -3    21860000       


## ***___# Current Liabilities___***

In [75]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:LiabilitiesCurrent$",
]).lower())

BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:liabilitiescurrent                                                       U_iso4217USD -3    509762000      


## Long Term Debt

In [76]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:LongTermLoansFromBank$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:longtermloansfrombank                                                    U_iso4217USD -3    288179000      


## Tax Deferred

In [77]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:DeferredIncomeTaxLiabilitiesNet$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:deferredincometaxliabilitiesnet                                          U_iso4217USD -3    34598000       


## Other Long Term Liabilities

In [78]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:OtherLiabilitiesNoncurrent$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:otherliabilitiesnoncurrent                                               U_iso4217USD -3    130795000      


## ***___# Total Liabilities___***

In [79]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:Liabilities$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:liabilities                                                              U_iso4217USD -3    963334000      


## Paid-in Capital

* [Paid-In Capital](https://www.investopedia.com/terms/p/paidincapital.aspPaid-In Capital)

> Paid-in capital represents the funds raised by the business through selling its equity and not from ongoing business operations. Paid-in capital also refers to a line item on the company's balance sheet listed under shareholders' equity (also referred to as stockholders' equity), often shown alongside the line item for additional paid-in capital.

* [Additional Paid-In Capital (APIC)](https://www.investopedia.com/terms/a/additionalpaidincapital.asp)

> Often referred to as "contributed capital in excess of par,” APIC occurs when an investor buys newly-issued shares directly from a company during its initial public offering (IPO) stage. APIC, which is itemized under the shareholder equity (SE) section of a balance sheet, is viewed as a profit opportunity for companies as it results in them receiving excess cash from stockholders.

In [80]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:AdditionalPaidInCapitalCommonStock$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:additionalpaidincapitalcommonstock                                       U_iso4217USD -3    449598000      


## Retained Earnings

* [Retained Earnings](https://www.investopedia.com/terms/r/retainedearnings.asp)

>  The word "retained" captures the fact that because those earnings were **NOT paid out to shareholders as dividends** they were instead retained by the company.

In [81]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:RetainedEarningsAppropriated$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:retainedearningsappropriated                                             U_iso4217USD -3    888046000      


## Accumulated other comprehensive loss

In [82]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:AccumulatedOtherComprehensiveIncomeLossNetOfTax$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:accumulatedothercomprehensiveincomelossnetoftax                          U_iso4217USD -3    -73606000      


## ***___# Stockholder's Equity___***

* [Stockholders' Equity](https://www.investopedia.com/terms/s/stockholdersequity.asp)

> Remaining amount of assets available to shareholders after all liabilities have been paid. (純資産)

In [83]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:StockholdersEquity$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:stockholdersequity                                                       U_iso4217USD -3    963820000      


## Interest

In [84]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:MinorityInterest$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:minorityinterest                                                         U_iso4217USD -3    52303000       


## ***___# Total Equity___***

In [85]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:stockholdersequityincludingportionattributabletononcontrollinginterest   U_iso4217USD -3    1016123000     


## ***___# Total Liabilities + Stockholder's Equity___***

In [86]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:LiabilitiesAndStockholdersEquity$",
]).lower())
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:liabilitiesandstockholdersequity                                         U_iso4217USD -3    1979457000     


## ***___# Shares Outstandings___***

In [87]:
BS += shares_outstandings

## Display B/S

In [88]:
df_BS = pd.DataFrame(BS, columns=get_financial_element_columns())
df_BS

,type,name,unit,decimals,value,context
0,credit,us-gaap:cashandcashequivalentsatcarryingvalue,U_iso4217USD,-3,268065000,C_0000029002_20201231
1,credit,us-gaap:restrictedcashequivalentscurrent,U_iso4217USD,-3,52464000,C_0000029002_20201231
2,credit,us-gaap:shortterminvestments,U_iso4217USD,-3,6142000,C_0000029002_20201231
3,credit,us-gaap:receivablesnetcurrent,U_iso4217USD,-3,320061000,C_0000029002_20201231
4,credit,us-gaap:inventorynet,U_iso4217USD,-3,307062000,C_0000029002_20201231
5,credit,us-gaap:prepaidexpenseandotherassetscurrent,U_iso4217USD,-3,70193000,C_0000029002_20201231
6,calc,us-gaap:assetscurrent,U_iso4217USD,-3,1023987000,C_0000029002_20201231
7,credit,us-gaap:propertyplantandequipmentnet,U_iso4217USD,-3,530815000,C_0000029002_20201231
8,credit,us-gaap:deferredincometaxassetsnet,U_iso4217USD,-3,57841000,C_0000029002_20201231
9,credit,us-gaap:goodwill,U_iso4217USD,-3,158331000,C_0000029002_20201231


In [89]:
credits = df_BS[df_BS['type'] == 'credit']['value'].sum()
credits

1979457000.0

In [90]:
debits = df_BS[df_BS['type'] == 'debit']['value'].sum()
debits

2139112000.0

In [91]:
credits - debits

-159655000.0

In [92]:
cps = np.divide(
    df_BS[df_BS['name'] == 'us-gaap:cashandcashequivalentsatcarryingvalue']['value'].values,
    df_BS[df_BS['name'] == 'us-gaap:commonstocksharesoutstanding']['value'].values
).item()
scale = 2

print(f"{cps:.{scale}f}")

6.05
